In [ ]:
import datetime

date = datetime.datetime.now()
print(f'Today is Happy{date: %A, %d, %m, %Y}.', '\n')

import os
import itertools
from glob import glob
import random 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from  matplotlib.ticker import PercentFormatter
import itertools
from statannotations.Annotator import Annotator

sns.set_context("paper", rc={"axes.titlesize":16,"axes.labelsize":14, "xtick.labelsize":12, "ytick.labelsize":12})   
sns.set_style('white', rc={'font.family': 'serif', 'font.serif': 'Times New Roman'})

### Button

In [ ]:
windows = False # True for Windows, False for MAC
stim_type = 'elec' # 'opt' or 'elec'
top1 = False # True for top1, False if top2
class_num = 16

if windows:
    base_dir = 'C:\\Users\\Na Min An\\Desktop\\Development\\ArtificialVision'
    slash = '\\'
else:
    base_dir = '/Volumes/Seagate/Development/ArtificialVision'
    slash = '/'

data_dir = os.path.join(base_dir, 'data')
data_path = os.path.join(data_dir, '210827_ANNA_Removing_uncontaminated_data.csv')
label_path = os.path.join(data_dir, '211105_QAs_for_Set0_CNN_SVC_4classes_partial.csv')
pred_dir =  os.path.join(os.path.join(data_dir, 'Human_Expert'), '211202')
quesdata_files = glob(f'{base_dir}{slash}data{slash}Question_Banks_AI_Hub_final{slash}16classes{slash}set0{slash}*')
fig_path = glob(f'{base_dir}{slash}figures')[0]

df = pd.read_csv(data_path)
l = list(range(df.shape[0]))
random.seed(22)
set_1 = random.sample(l, class_num)
sets = [set_1]
face_lst = [df.iloc[i, 0] for i in set_1]
age_lst = [df.iloc[i, 1].split('대')[0] for i in set_1]
gen_lst = [df.iloc[i, 2] for i in set_1]
gen_dic = {'남':'M', '여':'F'}
face_att_dic = {face_lst[i] : [age_lst[i], gen_dic[gen_lst[i]]] for i in range(len(face_lst))}
image_dir = f'{data_dir}{slash}sample_for_dev_{stim_type}'

Human data

In [ ]:
if stim_type == 'opt':
  sel_ppl = list(range(300, 309)) + list(range(400, 408)) + [611] # 18 subjects
elif stim_type == 'elec': 
  sel_ppl = [499, 500] + list(range(502, 509)) + list(range(601, 607)) + list(range(608, 611)) # 18 subjects
random.seed(42)

human_df = pd.DataFrame()
preds = []
n = 9
for i in range(1, 80*n+1, 80):
    j = i+79 
    temp_df = pd.read_csv(os.path.join(pred_dir, f'main_test({i}_{j}).csv'))
    temp_df = temp_df[temp_df['유저식별아이디'].isin(sel_ppl)]
    human_df = pd.concat([human_df, temp_df], axis=1)
    # To calculate the mode
    if top1:
      temp_df = temp_df.loc[:, temp_df.columns.str.startswith('선택_A')]
      temp_df = temp_df.fillna(0)
      for k in range(len(temp_df.columns)):
          temp_preds = temp_df.iloc[:, k].values.astype(str).astype(float).astype(int)
          temp_preds = temp_preds[temp_preds > 0]
          temp_pred = np.bincount(temp_preds).argmax() 
          preds.append(temp_pred)
    else:
      temp_df1 = temp_df.loc[:, temp_df.columns.str.startswith('선택_A')]
      temp_df2 = temp_df.loc[:, temp_df.columns.str.startswith('선택_B')]
      temp_df1, temp_df2 = temp_df1.fillna(0), temp_df2.fillna(0)
      for k in range(len(temp_df1.columns)):
          temp_preds1 = temp_df1.iloc[:, k].values.astype(str).astype(float).astype(int)
          temp_preds2 = temp_df2.iloc[:, k].values.astype(str).astype(float).astype(int)
          temp_preds = np.concatenate([temp_preds1, temp_preds2])
          temp_preds = temp_preds[temp_preds > 0]
          temp_pred = np.bincount(temp_preds).argmax() 
          preds.append(temp_pred)  
        
orig_human_df = human_df
human_df = human_df.fillna(0)

sel_col = []
for j in range(1, 80*n+1):
  if top1:
    temp_str = f'선택_A_{j}'
    sel_col.append(temp_str)
  else:
    temp_str1 = f'선택_A_{j}'
    temp_str2 = f'선택_B_{j}'
    sel_col.append(temp_str1)
    sel_col.append(temp_str2)

human_df = human_df[sel_col]
human_df.index = sel_ppl 

if top1:
  human_df.columns = list(range(80*n))
else:
  human_df.columns = list(range(2*80*n))

question_df = pd.read_csv(label_path)

if top1:
  human_df.columns = question_df['Answer']
else:
  human_df.columns = [ans for ans in question_df['Answer'] for _ in (0, 1)]

processed_human_df = human_df.copy()
for (j, ans_file) in enumerate(processed_human_df.columns):
    face, pix, gs, par = ans_file.split('.')[0].split('_')
    act_face = int(face)
    for i in range(processed_human_df.shape[0]):
        pred_face = int(processed_human_df.iloc[i, j])
        if act_face == pred_face:
            processed_human_df.iloc[i, j] = 1.0
        else:
            processed_human_df.iloc[i, j] = 0.0

if not top1:
  processed_human_df = processed_human_df.iloc[:, 0:1440:2] + processed_human_df.iloc[:, 1:1440:2]

processed_human_df_T = processed_human_df.transpose()
face_lst, hyp_lst, pix_lst, gs_lst, par_lst, ang_lst, exp_lst, age_lst, gen_lst = [], [], [], [], [], [], [], [], []
for (i, ans_file) in enumerate(processed_human_df_T.index):
  face, pix, gs, par = ans_file.split('.')[0].split('_')
  face_lst.append(face)
  hyp_lst.append(f'{pix}_{gs}')
  pix_lst.append(pix)
  gs_lst.append(gs)
  par_lst.append(par)
  ang_lst.append(par.split('C')[-1])
  exp_lst.append(par.split('C')[0].split('E')[-1])
  age_lst.append(face_att_dic[int(face)][0])
  gen_lst.append(face_att_dic[int(face)][1])
processed_human_df_T['Face'] = face_lst
processed_human_df_T['Hyp'] = hyp_lst
processed_human_df_T['PIX'] = pix_lst
processed_human_df_T['GS'] = gs_lst
processed_human_df_T['Par'] = par_lst
processed_human_df_T['Ang'] = ang_lst
processed_human_df_T['Exp'] = exp_lst
processed_human_df_T['Age'] = age_lst
processed_human_df_T['Gen'] = gen_lst
processed_human_df_T['HumMean'] = processed_human_df_T[sel_ppl].mean(axis=1)
processed_human_df_T['HumMode'] = preds
temp_lst = list(map(int, (processed_human_df_T['HumMode'].astype(int) == processed_human_df_T['Face'].astype(int)).tolist()))
processed_human_df_T['HumMode'] = temp_lst

processed_human_df_T

Model data

In [ ]:
model_type1_list = [''] # ['PCA', 'PCA', '', '']
model_type2_list = ['CNN_SVC'] # ['SVC', 'LR', 'CNN_LR', 'CNN_SVC']
seed_list = [22, 77, 2, 100, 81, 42, 7, 1, 55, 50] 
pix_order_list = ['16PIX', '24PIX', '32PIX', '64PIX', '128PIX']
gs_order_list = ['2GS', '4GS', '6GS', '8GS', '16GS']

for (model_type1, model_type2) in zip(model_type1_list, model_type2_list): 
    model_type = model_type1 + model_type2
    input_folder = [df.iloc[i, 0] for i in sets[0]] 
    assert len(input_folder) == 16
    com_obj = itertools.combinations(input_folder, 16)
    com_list = list(com_obj)

    mac_df = pd.DataFrame()
    for i in range(len(quesdata_files)):
        quesdata_path = quesdata_files[i]
        seed = int(os.path.basename(quesdata_path).split('d')[-1])

        for n in range(len(os.listdir(quesdata_path))):
            print(seed, n)

            preprocessed_data_path =  os.path.join(quesdata_path, f'comb{n}') 
            high_analysis_path = os.path.join(preprocessed_data_path, f'High_Analysis_{stim_type}')
            
            add_high_df = pd.read_csv(os.path.join(high_analysis_path, f'High_Level_Data_Analysis_{model_type1}_{model_type2}.csv'))
            add_high_df['Hit Rate'] = add_high_df['correctness'].replace(['correct', 'wrong'], [1, 0]) 
            add_high_df = add_high_df[['file_name', 'actual_person', 'Hit Rate']]
            add_high_df['actual_person'] = add_high_df['actual_person'].astype(int)
            
            img_list, hyperpar_list, par_list = [], [], []
            for i in range(add_high_df.shape[0]):
                file_name = add_high_df['file_name'][i].split('.')[0]
                pix, gs, par = file_name.split('_')
                per = add_high_df['actual_person'][i]
                img = str(per) + '_' + str(pix) + '_' + str(gs) + '_' + str(par) + '.jpg'
                img_list.append(img)
                hyperpar_list.append(str(pix) + '_' + str(gs))
                par_list.append(str(par))
            add_high_df['img'] = img_list
            add_high_df['hyperpar'] = hyperpar_list
            add_high_df['par'] = par_list
            
            add_high_df = add_high_df[['img', 'hyperpar', 'par', 'Hit Rate']]
        
        add_high_df['Seed'] = [seed] * add_high_df.shape[0]
        
        mac_df = pd.concat([mac_df, add_high_df], axis=0)

new_hyperpar_name_list = ['16PIX_2GS', '16PIX_4GS', '16PIX_8GS', 
                          '32PIX_2GS', '32PIX_4GS', '32PIX_8GS',  
                          '64PIX_2GS', '64PIX_4GS', '64PIX_8GS']
par_list = ['S001L1E01C4', 'S001L1E01C7', 'S001L1E01C10',
            'S001L1E02C7',
            'S001L1E03C7']
mac_df = mac_df[mac_df['hyperpar'].isin(new_hyperpar_name_list)]
mac_df = mac_df[mac_df['par'].isin(par_list)]

mac_df = mac_df.pivot(index='Seed', columns='img', values='Hit Rate')
mac_df_T = mac_df.T
mac_df_T['ModMean'] = mac_df_T[seed_list].mean(axis=1)

mac_df_T

Merged data

In [ ]:
mer_df = pd.merge(processed_human_df_T, mac_df_T, left_index=True, right_index=True)
mer_df = mer_df[['HumMean', 'ModMean', 'Hyp', 'Par', 'Ang', 'Exp', 'Age', 'Gen']]

hum_df = mer_df[['HumMean', 'Hyp', 'Par', 'Ang', 'Exp', 'Age', 'Gen']]
mod_df = mer_df[['ModMean', 'Hyp', 'Par', 'Ang', 'Exp', 'Age', 'Gen']]

hum_df['HvsM'] = ['Hum'] * hum_df.shape[0]
mod_df['HvsM'] = ['Mod'] * mod_df.shape[0]

hum_df.columns = ['Mean', 'Hyp', 'Par', 'Ang', 'Exp', 'Age', 'Gen', 'HvsM']
mod_df.columns = ['Mean', 'Hyp', 'Par', 'Ang', 'Exp', 'Age', 'Gen', 'HvsM']

mer_df = pd.concat([hum_df, mod_df], axis=0)

mer_df

In [ ]:
mer_df.to_csv((os.path.join(fig_path, f'faceConAgeGen_{stim_type}.csv')))

Results

In [ ]:
# Is there a facial condition effect?

order = ['S001L1E01C10',
         'S001L1E01C4',
         'S001L1E01C7',
         'S001L1E02C7',
         'S001L1E03C7',]
g = sns.catplot(x='Par', y='Mean', col='HvsM', data=mer_df,
                order=order,
                kind='bar', aspect=1, capsize=0.1, errwidth=0.5,
                palette=sns.color_palette('Spectral', len(order)))
g.set_axis_labels('', 'Facial Recognition Accuracy (%)')
g.set_xticklabels(['Right profile',
                   'Left profile',
                   'Neutral front face',
                   'Joyful face',
                   'Sullen face'], rotation=45)
g.set_titles('{col_name}')
g.set(ylim=(0, 1))

for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(PercentFormatter(1))

pairs = list(itertools.combinations(order, 2))
args = dict(x='Par', y='Mean', col='HvsM', data=mer_df, order=order)
hvsm_dict = {0:'Hum', 1:'Mod'}

for ax_n in g.axes:
    for (i, ax) in enumerate(ax_n):
        args = dict(x='Par', y='Mean', col='HvsM', data=mer_df[mer_df['HvsM']==hvsm_dict[i]], order=order)
        annot = Annotator(ax, pairs, **args)
        annot.configure(test='Mann-Whitney', text_format='star', loc='outside')
        annot.apply_test().annotate()

plt.show()

In [ ]:
# Is there a gender effect?

order = ['F', 'M']
g = sns.catplot(x='Gen', y='Mean', col='HvsM', data=mer_df,
                order=order,
                kind='bar', aspect=0.5, capsize=0.1, errwidth=0.5,
                palette=sns.color_palette('hls', len(order)))
g.set_axis_labels('', 'Facial Recognition Accuracy (%)')
g.set_xticklabels(order)
g.set_titles('{col_name}')
g.set(ylim=(0, 1))

for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(PercentFormatter(1))

pairs = list(itertools.combinations(order, 2))
args = dict(x='Gen', y='Mean', col='HvsM', data=mer_df, order=order)
hvsm_dict = {0:'Hum', 1:'Mod'}

for ax_n in g.axes:
    for (i, ax) in enumerate(ax_n):
        args = dict(x='Gen', y='Mean', col='HvsM', data=mer_df[mer_df['HvsM']==hvsm_dict[i]], order=order)
        annot = Annotator(ax, pairs, **args)
        annot.configure(test='Mann-Whitney', text_format='star', loc='outside')
        annot.apply_test().annotate()

plt.show()

In [ ]:
# Is there an gender effect?

order = ['30', '40', '50']
g = sns.catplot(x='Age', y='Mean', col='HvsM', data=mer_df,
                order=order,
                kind='bar', aspect=0.5, capsize=0.1, errwidth=0.5,
                palette=sns.light_palette('seagreen', len(order)))
g.set_axis_labels('', 'Facial Recognition Accuracy (%)')
g.set_xticklabels(order)
g.set_titles('{col_name}')
g.set(ylim=(0, 1))

for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(PercentFormatter(1))

pairs = list(itertools.combinations(order, 2))
hvsm_dict = {0:'Hum', 1:'Mod'}

for ax_n in g.axes:
    for (i, ax) in enumerate(ax_n):
        args = dict(x='Age', y='Mean', col='HvsM', data=mer_df[mer_df['HvsM']==hvsm_dict[i]], order=order)
        annot = Annotator(ax, pairs, **args)
        annot.configure(test='Mann-Whitney', text_format='star', loc='outside')
        annot.apply_test().annotate()

plt.show()

Miscallaneous

In [ ]:
# Is there a visual angle effect?

order = ['16PIX_2GS', '16PIX_4GS', '16PIX_8GS',
         '32PIX_2GS', '32PIX_4GS', '32PIX_8GS',
         '64PIX_2GS', '64PIX_4GS', '64PIX_8GS']
sns.catplot(x='Hyp', y='Mean', hue='HvsM', data=mer_df,
            order=order, 
            kind='bar')
plt.xticks(rotation=45)
plt.ylabel('Facial Recognition Accuracy')
plt.show()

In [ ]:
# Is there a visual angle effect?

order = ['16PIX_2GS', '16PIX_4GS', '16PIX_8GS',
         '32PIX_2GS', '32PIX_4GS', '32PIX_8GS',
         '64PIX_2GS', '64PIX_4GS', '64PIX_8GS']
hue_order = ['4', '7','10']
sns.catplot(x='Hyp', y='Mean', col='HvsM', hue='Ang', data=mer_df,
            palette=sns.color_palette("hls", len(hue_order)),
            order=order, hue_order=hue_order,
            kind='bar')
plt.xticks(rotation=45)
plt.ylabel('Facial Recognition Accuracy')
plt.show()

In [ ]:
# Is there a facial expression effect?

order = ['16PIX_2GS', '16PIX_4GS', '16PIX_8GS',
         '32PIX_2GS', '32PIX_4GS', '32PIX_8GS',
         '64PIX_2GS', '64PIX_4GS', '64PIX_8GS']
hue_order = ['01', '02', '03']
sns.catplot(x='Hyp', y='Mean', col='HvsM', hue='Exp', data=mer_df,
            palette=sns.color_palette("hls", len(hue_order)),
            order=order, hue_order=hue_order,
            kind='bar')
plt.xticks(rotation=45)
plt.ylabel('Facial Recognition Accuracy')
plt.show()

In [ ]:
# Is there an age effect?

order = ['16PIX_2GS', '16PIX_4GS', '16PIX_8GS',
         '32PIX_2GS', '32PIX_4GS', '32PIX_8GS',
         '64PIX_2GS', '64PIX_4GS', '64PIX_8GS']
hue_order = ['30', '40', '50']
sns.catplot(x='Hyp', y='Mean', col='HvsM', hue='Age', data=mer_df,
            palette=sns.color_palette("hls", len(hue_order)),
            order=order, hue_order=hue_order,
            kind='bar')
plt.xticks(rotation=45)
plt.ylabel('Facial Recognition Accuracy')
plt.show()

In [ ]:
# Is there a gender effect?

order = ['16PIX_2GS', '16PIX_4GS', '16PIX_8GS',
         '32PIX_2GS', '32PIX_4GS', '32PIX_8GS',
         '64PIX_2GS', '64PIX_4GS', '64PIX_8GS']
hue_order = ['F', 'M']
sns.catplot(x='Hyp', y='Mean', col='HvsM', hue='Gen', data=mer_df,
            palette=sns.color_palette("hls", len(hue_order)),
            order=order, hue_order=hue_order,
            kind='bar')
plt.xticks(rotation=45)
plt.ylabel('Facial Recognition Accuracy')
plt.show()